In [1]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import re
import time
import collections
import os

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def build_dataset(words, n_words, atleast=1):
    count = [['GO', 0], ['PAD', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [3]:
lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]
        
convs = [ ]
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))
    
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])
        
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    return ' '.join([i.strip() for i in filter(None, text.split())])

clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))
    
min_line_length = 2
max_line_length = 5
short_questions_temp = []
short_answers_temp = []

i = 0
for question in clean_questions:
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])
    i += 1

short_questions = []
short_answers = []

i = 0
for answer in short_answers_temp:
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
    i += 1

question_test = short_questions[500:550]
answer_test = short_answers[500:550]
short_questions = short_questions[:500]
short_answers = short_answers[:500]

In [4]:
concat_from = ' '.join(short_questions+question_test).split()
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])
print('filtered vocab size:',len(dictionary_from))
print("% of vocab used: {}%".format(round(len(dictionary_from)/vocabulary_size_from,4)*100))

vocab from size: 657
Most common words [('you', 132), ('is', 78), ('i', 68), ('what', 51), ('it', 50), ('that', 49)]
Sample data [7, 28, 205, 40, 81, 44, 12, 23, 87, 535] ['what', 'good', 'stuff', 'she', 'okay', 'they', 'do', 'to', 'hey', 'sweet']
filtered vocab size: 661
% of vocab used: 100.61%


In [5]:
concat_to = ' '.join(short_answers+answer_test).split()
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab from size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])
print('filtered vocab size:',len(dictionary_to))
print("% of vocab used: {}%".format(round(len(dictionary_to)/vocabulary_size_to,4)*100))

vocab from size: 660
Most common words [('i', 97), ('you', 91), ('is', 62), ('it', 58), ('not', 47), ('what', 39)]
Sample data [12, 590, 5, 4, 97, 25, 60, 10, 8, 86] ['the', 'real', 'you', 'i', 'hope', 'so', 'they', 'do', 'not', 'hi']
filtered vocab size: 664
% of vocab used: 100.61%


In [6]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [7]:
for i in range(len(short_answers)):
    short_answers[i] += ' EOS'

In [8]:
class Chatbot:
    def __init__(self, size_layer, num_layers, embedded_size,
                 from_dict_size, to_dict_size, learning_rate, batch_size):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),
                                           reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.placeholder(tf.int32, [None])
        self.Y_seq_len = tf.placeholder(tf.int32, [None])
        
        encoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embeddings = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        main = tf.strided_slice(self.X, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        decoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, decoder_input)
        
        attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_units = size_layer, 
                                                                    memory = encoder_embedded)
        rnn_cells = tf.contrib.seq2seq.AttentionWrapper(cell = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)]), 
                                                        attention_mechanism = attention_mechanism,
                                                        attention_layer_size = size_layer)
        _, last_state = tf.nn.dynamic_rnn(rnn_cells, encoder_embedded,
                                          dtype = tf.float32)
        last_state = tuple(last_state[0][-1] for _ in range(num_layers))
        with tf.variable_scope("decoder"):
            rnn_cells_dec = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
            outputs, _ = tf.nn.dynamic_rnn(rnn_cells_dec, decoder_embedded, 
                                           initial_state = last_state,
                                           dtype = tf.float32)
        self.logits = tf.layers.dense(outputs,to_dict_size)
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)

In [9]:
size_layer = 256
num_layers = 2
embedded_size = 128
learning_rate = 0.001
batch_size = 16
epoch = 20

In [10]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Chatbot(size_layer, num_layers, embedded_size, len(dictionary_from), 
                len(dictionary_to), learning_rate,batch_size)
sess.run(tf.global_variables_initializer())

In [11]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            try:
                ints.append(dic[k])
            except Exception as e:
                print(e)
                ints.append(UNK)
        X.append(ints)
    return X

In [12]:
X = str_idx(short_questions, dictionary_from)
Y = str_idx(short_answers, dictionary_to)
X_test = str_idx(question_test, dictionary_from)
Y_test = str_idx(answer_test, dictionary_from)

'hallucinating'
'hobbs'
'having'
'nightmare'
'fivepointed'
'star'
'freezing'
'pecos'
'circling'
'walk'
'wanted'
'telephone'
'suppose'
'guinness'
'suffice'
'twentyfour'
'clark'
'apartment'
'misunderstood'
'bachelor'
'ummm'
'eh'
'sampsongs'
'office'
'send'
'protecting'
'somebody'
'mine'
'charlie'


In [13]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = 10
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(10)
    return padded_seqs, seq_lens

def check_accuracy(logits, Y):
    acc = 0
    for i in range(logits.shape[0]):
        internal_acc = 0
        count = 0
        for k in range(len(Y[i])):
            try:
                if Y[i][k] == logits[i][k]:
                    internal_acc += 1
                count += 1
                if Y[i][k] == EOS:
                    break
            except:
                break
        acc += (internal_acc / count)
    return acc / logits.shape[0]

In [14]:
for i in range(epoch):
    total_loss, total_accuracy = 0, 0
    for k in range(0, (len(short_questions) // batch_size) * batch_size, batch_size):
        batch_x, seq_x = pad_sentence_batch(X[k: k+batch_size], PAD)
        batch_y, seq_y = pad_sentence_batch(Y[k: k+batch_size], PAD)
        predicted, loss, _ = sess.run([tf.argmax(model.logits,2), model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y,
                                                model.X_seq_len:seq_x,
                                                model.Y_seq_len:seq_y})
        total_loss += loss
        total_accuracy += check_accuracy(predicted,batch_y)
    total_loss /= (len(short_questions) // batch_size)
    total_accuracy /= (len(short_questions) // batch_size)
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1, total_loss, total_accuracy))

epoch: 1, avg loss: 3.007761, avg accuracy: 0.112534
epoch: 2, avg loss: 2.325608, avg accuracy: 0.167944
epoch: 3, avg loss: 2.262170, avg accuracy: 0.173790
epoch: 4, avg loss: 2.227036, avg accuracy: 0.183266
epoch: 5, avg loss: 2.196803, avg accuracy: 0.179671
epoch: 6, avg loss: 2.166067, avg accuracy: 0.179234
epoch: 7, avg loss: 2.136252, avg accuracy: 0.178931
epoch: 8, avg loss: 2.107442, avg accuracy: 0.180914
epoch: 9, avg loss: 2.079889, avg accuracy: 0.198219
epoch: 10, avg loss: 2.052072, avg accuracy: 0.219086
epoch: 11, avg loss: 2.022121, avg accuracy: 0.232124
epoch: 12, avg loss: 1.997891, avg accuracy: 0.232426
epoch: 13, avg loss: 1.977490, avg accuracy: 0.237097
epoch: 14, avg loss: 1.973568, avg accuracy: 0.240255
epoch: 15, avg loss: 1.995836, avg accuracy: 0.224530
epoch: 16, avg loss: 1.953274, avg accuracy: 0.226411
epoch: 17, avg loss: 1.880463, avg accuracy: 0.247917
epoch: 18, avg loss: 1.847609, avg accuracy: 0.255612


ResourceExhaustedError: OOM when allocating tensor with shape[512,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: gradients/decoder/rnn/while/rnn/multi_rnn_cell/cell_1/lstm_cell/MatMul_grad/MatMul_1 = MatMul[T=DT_FLOAT, transpose_a=true, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](gradients/decoder/rnn/while/rnn/multi_rnn_cell/cell_1/lstm_cell/MatMul_grad/MatMul_1/StackPopV2, gradients/decoder/rnn/while/rnn/multi_rnn_cell/cell_1/lstm_cell/BiasAdd_grad/tuple/control_dependency)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'gradients/decoder/rnn/while/rnn/multi_rnn_cell/cell_1/lstm_cell/MatMul_grad/MatMul_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    ex

In [15]:
for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
QUESTION: you making any headway
REAL ANSWER: she kissed me
PREDICTED ANSWER: i is 

row 2
QUESTION: that is the worst
REAL ANSWER: you get the girl
PREDICTED ANSWER: i is 

row 3
QUESTION: hey do you mind
REAL ANSWER: not at all
PREDICTED ANSWER: i is 

row 4
QUESTION: a hundred bucks a date
REAL ANSWER: forget it
PREDICTED ANSWER: i is 

row 5
QUESTION: forget it
REAL ANSWER: forget her sister then
PREDICTED ANSWER: i is 

row 6
QUESTION: it is about time
REAL ANSWER: a deal's a deal
PREDICTED ANSWER: i is 

row 7
QUESTION: how would you do it
REAL ANSWER: do what
PREDICTED ANSWER: i is 

row 8
QUESTION: who's that
REAL ANSWER: patrick verona random skid
PREDICTED ANSWER: i is 

row 9
QUESTION: he always look so
REAL ANSWER: block e
PREDICTED ANSWER: i is not 

row 10
QUESTION: that is this
REAL ANSWER: an attempted slit
PREDICTED ANSWER: i is not 

row 11
QUESTION: that is never been proven
REAL ANSWER: neither has his heterosexuality
PREDICTED ANSWER: i is 

row 12
QUESTION: 

In [16]:
batch_x, seq_x = pad_sentence_batch(X_test[:batch_size], PAD)
batch_y, seq_y = pad_sentence_batch(Y_test[:batch_size], PAD)
predicted = sess.run(tf.argmax(model.logits,2), feed_dict={model.X:batch_x,model.X_seq_len:seq_x})

for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
QUESTION: but david
REAL ANSWER: is be that
PREDICTED ANSWER: i is 

row 2
QUESTION: hopeless it is hopeless
REAL ANSWER: chief mean my sure
PREDICTED ANSWER: i is a 

row 3
QUESTION: miss price
REAL ANSWER: me late
PREDICTED ANSWER: i is 

row 4
QUESTION: mr kessler wake up please
REAL ANSWER: is be are
PREDICTED ANSWER: i is 

row 5
QUESTION: there were witnesses
REAL ANSWER: thank she said
PREDICTED ANSWER: i is not it 

row 6
QUESTION: what about it
REAL ANSWER: not you are
PREDICTED ANSWER: i is 

row 7
QUESTION: go on ask them
REAL ANSWER: i at czech
PREDICTED ANSWER: i is 

row 8
QUESTION: beware the moon
REAL ANSWER: missed ask is was
PREDICTED ANSWER: i is 

row 9
QUESTION: did you hear that
REAL ANSWER: is girl what
PREDICTED ANSWER: i is 

row 10
QUESTION: i heard that
REAL ANSWER: it be not
PREDICTED ANSWER: i is not 

row 11
QUESTION: the hound of the baskervilles
REAL ANSWER: bad
PREDICTED ANSWER: i is 

row 12
QUESTION: it is moving
REAL ANSWER: not you remember
PR